In [6]:
from selenium import webdriver
import pickle
import time

# Launch browser
driver = webdriver.Chrome()
driver.get("https://dbv.turnier.de/user")  # Login page

input("🔐 Please log in manually, then press Enter...")

# Save cookies to file
with open("cookies.pkl", "wb") as f:
    pickle.dump(driver.get_cookies(), f)

print("✅ Session cookies saved to 'cookies.pkl'.")
driver.quit()


🔐 Please log in manually, then press Enter... 2094yym@gmail.com


✅ Session cookies saved to 'cookies.pkl'.


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import time

def get_player_rankings(name_list):
    results = {}

    for player_name in name_list:
        print("#########################")
        print("current player :", player_name)

        # 🔁 Start new browser session for each player
        driver = webdriver.Chrome()
        driver.get("https://dbv.turnier.de")  # Initial load to set domain

        # 🔐 Load cookies into the new session
        try:
            with open("cookies.pkl", "rb") as f:
                cookies = pickle.load(f)
                for cookie in cookies:
                    driver.add_cookie(cookie)
            print("✅ Cookies loaded.")
        except Exception as e:
            print("❌ Failed to load cookies:", e)
            driver.quit()
            results[player_name] = None
            continue

        driver.get("https://dbv.turnier.de")  # Reload to apply cookies
        time.sleep(1)

        try:
            print(f"\n🔍 Searching for: {player_name}")
            search_url = f"https://dbv.turnier.de/find/player?q={player_name}"
            driver.get(search_url)

            # Optional quick cookie/language button handling
            try:
                button = driver.find_elements(By.XPATH, '/html/body/div/div/div/main/form/div[1]/button[1]/span')
                if button:
                    button[0].click()
                    print("✅ Clicked language/cookie accept button.")
                    time.sleep(1)
            except:
                print("⚠️ No cookie/language button found.")

            # Wait for search results
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "searchResultArea"))
            )

            # Look for exact name match
            spans = driver.find_elements(By.XPATH, '//div[@class="media__content"]/h5/a/span[@class="nav-link__value"]')
            profile_link = None
            for span in spans:
                if span.text.strip().lower() == player_name.lower():
                    profile_link = span.find_element(By.XPATH, "./ancestor::a").get_attribute("href")
                    break

            if not profile_link:
                print(f"❌ Exact match not found: {player_name}")
                results[player_name] = None
                driver.quit()
                continue

            print(f"✅ Found profile: {profile_link}")
            driver.get(profile_link)

            # Wait for profile to load
            WebDriverWait(driver, 10).until(EC.url_contains("/player-profile/"))
            print(f"🌐 Opened profile page for {player_name}")

            # Click on 'Ranglisten' tab
            # Wait up to 5 seconds to find the tab
            try:
                rank_tabs = driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/ul/li[4]/a')
                if rank_tabs:
                    print("✅ 'Ranglisten' tab found.")
                    try:
                        driver.execute_script("arguments[0].click();", rank_tabs[0])
                        print("✅ Clicked on 'Ranglisten' tab (via JS).")
                    except Exception as e:
                        print(f"❌ Failed to click 'Ranglisten' tab via JS: {e}")
                        results[player_name] = None
                        driver.quit()
                        continue
                else:
                    print("❌ 'Ranglisten' tab not found.")
                    results[player_name] = None
                    driver.quit()
                    continue
            except Exception as e:
                print(f"❌ Error during 'Ranglisten' tab handling: {e}")
                results[player_name] = None
                driver.quit()
                continue


            # Parse ranking table
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//table//tr[.//th[@class="th__title"]]'))
            )

            rows = driver.find_elements(By.XPATH, '//table//tr[.//th[@class="th__title"]]')
            rankings = {}

            for row in rows:
                try:
                    try:
                        event = row.find_element(By.XPATH, './th/a').text.strip()
                    except:
                        event = row.find_element(By.XPATH, './th').text.strip()

                    if event.lower() == "disziplin":
                        continue

                    tds = row.find_elements(By.TAG_NAME, 'td')
                    if not tds:
                        continue

                    rank = tds[0].text.strip()
                    rankings[event] = rank
                except Exception as e:
                    print(f"⚠️ Skipping row: {e}")

            if rankings:
                print(f"📊 Rankings Extracted for {player_name}:")
                for k, v in rankings.items():
                    print(f"  {k}: {v}")
                results[player_name] = rankings
            else:
                print("⚠️ No rankings found.")
                results[player_name] = {}

        except Exception as e:
            print(f"❌ Error processing {player_name}:", e)
            results[player_name] = None

        driver.quit()  # 🧼 Always close driver

    return results


In [9]:
players = [
    "Yumeng Yan", "Sharon Yeh",
    "Mortiz Kasten", "Carolin Weiß",
    "Felix Bothe", "Julia Lampe",
    "Sven Zahn", "Jacqueline Lobscheid",
    "Tobias Schneider", "Emilia Wegener",
    "Fabian Winkler", "Lisa Heidel",
    "Claudius Stolle", "Nina Stolle",
    "Francis Barg", "Sophia Quante",
    "Benjamin Möller", "Hanna Lenke",
    "Sebastian Kretschmer", "Enrica Wegener",
    "Dennis Lieser", "Christina Lieser",
    "Ruven Rippe", "Merle Matheja",
    "Christoph Herda", "Kira Brandenburg",
    "Oriz Putra", "Marit Fisahn"
]


In [10]:

all_rankings = get_player_rankings(players)

print("\n🎯 Final Result:")
for name, ranks in all_rankings.items():
    print(f"{name}: {ranks}")


#########################
current player : Yumeng Yan
✅ Cookies loaded.

🔍 Searching for: Yumeng Yan
✅ Clicked language/cookie accept button.
✅ Found profile: https://dbv.turnier.de/player-profile/B0DD5E01-8E0A-4D56-8E72-6D71A12FEBE6
🌐 Opened profile page for Yumeng Yan
✅ 'Ranglisten' tab found.
✅ Clicked on 'Ranglisten' tab (via JS).
📊 Rankings Extracted for Yumeng Yan:
  HE: 2503
  HD: 3405
#########################
current player : Sharon Yeh
✅ Cookies loaded.

🔍 Searching for: Sharon Yeh
✅ Clicked language/cookie accept button.
❌ Exact match not found: Sharon Yeh
#########################
current player : Mortiz Kasten
✅ Cookies loaded.

🔍 Searching for: Mortiz Kasten
✅ Clicked language/cookie accept button.
❌ Exact match not found: Mortiz Kasten
#########################
current player : Carolin Weiß
✅ Cookies loaded.

🔍 Searching for: Carolin Weiß
✅ Clicked language/cookie accept button.
✅ Found profile: https://dbv.turnier.de/player-profile/48A31589-D2C5-4042-B318-6EE3B873ACFA
🌐